<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/multi_agent_comm_via_mcp/multi_agent_mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent System with MCP (Model Context Protocol)

This notebook demonstrates how to build a multi-agent system where agents communicate using the **Model Context Protocol (MCP)**.

## What is MCP?

The Model Context Protocol is an open protocol developed by Anthropic that standardizes how applications provide context to LLMs. It enables:
- **Standardized tool interfaces** between agents
- **Resource sharing** (data, context, state)
- **Prompts and templates** that can be shared
- **Interoperability** between different AI systems

## Architecture

We'll build a system where:
1. **MCP Servers** expose tools and resources
2. **Agent MCP Clients** consume these tools
3. **Agents communicate** by sharing MCP resources
4. **Coordinator** orchestrates the multi-agent workflow

## Installation

```bash
pip install mcp anthropic httpx asyncio
```

In [ ]:
# Import required libraries
import asyncio
import json
from typing import Any, Dict, List, Optional
from dataclasses import dataclass, field
from datetime import datetime
import uuid

In [ ]:
# Install MCP SDK if needed (uncomment to run)
# !pip install mcp anthropic

## Part 1: MCP Server Implementation

First, we'll create MCP servers that expose tools and resources for agents to use.

In [ ]:
from mcp.server import Server
from mcp.types import Tool, Resource, TextContent, ImageContent, EmbeddedResource
from mcp.server.stdio import stdio_server

# Shared message bus for inter-agent communication
class MessageBus:
    """Simulates an MCP-based message bus for agent communication"""
    def __init__(self):
        self.messages: List[Dict[str, Any]] = []
        self.resources: Dict[str, Any] = {}
    
    def publish(self, agent_id: str, message_type: str, content: Any):
        """Publish a message to the bus"""
        msg = {
            "id": str(uuid.uuid4()),
            "agent_id": agent_id,
            "type": message_type,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        self.messages.append(msg)
        print(f"📤 [{agent_id}] Published: {message_type}")
        return msg["id"]
    
    def subscribe(self, agent_id: str, message_type: Optional[str] = None) -> List[Dict]:
        """Subscribe to messages from the bus"""
        if message_type:
            filtered = [m for m in self.messages 
                       if m["type"] == message_type and m["agent_id"] != agent_id]
        else:
            filtered = [m for m in self.messages if m["agent_id"] != agent_id]
        return filtered
    
    def store_resource(self, resource_id: str, data: Any):
        """Store a shared resource"""
        self.resources[resource_id] = data
        print(f"💾 Stored resource: {resource_id}")
    
    def get_resource(self, resource_id: str) -> Optional[Any]:
        """Retrieve a shared resource"""
        return self.resources.get(resource_id)

# Global message bus
message_bus = MessageBus()

## Part 2: MCP-Based Agent Implementation

Each agent acts as both an MCP client (consuming tools) and server (exposing capabilities).

In [ ]:
@dataclass
class MCPAgent:
    """An agent that communicates via MCP protocol"""
    agent_id: str
    role: str
    capabilities: List[str] = field(default_factory=list)
    message_bus: MessageBus = field(default_factory=lambda: message_bus)
    
    def __post_init__(self):
        # Register agent on the message bus
        self.message_bus.publish(
            self.agent_id, 
            "agent_registered",
            {"role": self.role, "capabilities": self.capabilities}
        )
    
    def expose_tool(self, tool_name: str, tool_func) -> Tool:
        """Expose a tool via MCP protocol"""
        return Tool(
            name=f"{self.agent_id}_{tool_name}",
            description=f"Tool from {self.agent_id}: {tool_name}",
            inputSchema={
                "type": "object",
                "properties": {},
            }
        )
    
    def publish_result(self, result_type: str, data: Any):
        """Publish results to other agents via MCP"""
        return self.message_bus.publish(self.agent_id, result_type, data)
    
    def get_messages(self, message_type: Optional[str] = None) -> List[Dict]:
        """Get messages from other agents"""
        return self.message_bus.subscribe(self.agent_id, message_type)
    
    def share_resource(self, resource_name: str, data: Any):
        """Share a resource with other agents"""
        resource_id = f"{self.agent_id}_{resource_name}"
        self.message_bus.store_resource(resource_id, data)
        self.publish_result("resource_shared", {"resource_id": resource_id})
        return resource_id
    
    def access_resource(self, resource_id: str) -> Optional[Any]:
        """Access a shared resource"""
        return self.message_bus.get_resource(resource_id)
    
    async def process_task(self, task: str) -> Dict[str, Any]:
        """Process a task - to be overridden by specific agents"""
        raise NotImplementedError("Subclasses must implement process_task")

## Part 3: Specialized Agents with MCP Communication

Let's create specialized agents that communicate via MCP.

In [ ]:
class DataCollectorAgent(MCPAgent):
    """Agent that collects and shares data"""
    
    def __init__(self):
        super().__init__(
            agent_id="data_collector",
            role="Data Collection",
            capabilities=["web_scraping", "api_calls", "data_extraction"]
        )
    
    async def process_task(self, task: str) -> Dict[str, Any]:
        """Simulate data collection"""
        print(f"\n🔍 [{self.agent_id}] Collecting data for: {task}")
        
        # Simulate data collection
        await asyncio.sleep(0.5)
        
        data = {
            "task": task,
            "data_points": [10, 25, 30, 45, 50],
            "source": "simulated_api",
            "timestamp": datetime.now().isoformat()
        }
        
        # Share data via MCP resource
        resource_id = self.share_resource("collected_data", data)
        
        # Publish completion message
        self.publish_result("data_collected", {
            "resource_id": resource_id,
            "summary": f"Collected {len(data['data_points'])} data points"
        })
        
        return data


class AnalysisAgent(MCPAgent):
    """Agent that analyzes data from other agents"""
    
    def __init__(self):
        super().__init__(
            agent_id="analyzer",
            role="Data Analysis",
            capabilities=["statistical_analysis", "pattern_recognition", "insights"]
        )
    
    async def process_task(self, task: str) -> Dict[str, Any]:
        """Analyze data from other agents"""
        print(f"\n📊 [{self.agent_id}] Analyzing data for: {task}")
        
        # Check for data collection messages
        messages = self.get_messages("data_collected")
        
        if not messages:
            print(f"⚠️  No data available for analysis")
            return {"error": "No data available"}
        
        # Get the latest data resource
        latest_msg = messages[-1]
        resource_id = latest_msg["content"]["resource_id"]
        data = self.access_resource(resource_id)
        
        print(f"📥 Retrieved resource: {resource_id}")
        
        # Simulate analysis
        await asyncio.sleep(0.5)
        
        analysis = {
            "task": task,
            "data_source": resource_id,
            "mean": sum(data["data_points"]) / len(data["data_points"]),
            "max": max(data["data_points"]),
            "min": min(data["data_points"]),
            "insights": "Data shows an upward trend",
            "timestamp": datetime.now().isoformat()
        }
        
        # Share analysis results
        resource_id = self.share_resource("analysis_results", analysis)
        
        # Publish completion
        self.publish_result("analysis_complete", {
            "resource_id": resource_id,
            "summary": f"Mean: {analysis['mean']:.2f}, Trend: Upward"
        })
        
        return analysis


class ReportGeneratorAgent(MCPAgent):
    """Agent that generates reports from analysis"""
    
    def __init__(self):
        super().__init__(
            agent_id="report_generator",
            role="Report Generation",
            capabilities=["report_writing", "visualization", "documentation"]
        )
    
    async def process_task(self, task: str) -> Dict[str, Any]:
        """Generate report from analysis"""
        print(f"\n📝 [{self.agent_id}] Generating report for: {task}")
        
        # Check for analysis completion messages
        messages = self.get_messages("analysis_complete")
        
        if not messages:
            print(f"⚠️  No analysis available for report")
            return {"error": "No analysis available"}
        
        # Get the analysis resource
        latest_msg = messages[-1]
        resource_id = latest_msg["content"]["resource_id"]
        analysis = self.access_resource(resource_id)
        
        print(f"📥 Retrieved resource: {resource_id}")
        
        # Simulate report generation
        await asyncio.sleep(0.5)
        
        report = {
            "title": f"Analysis Report: {task}",
            "summary": f"Analysis shows data with mean of {analysis['mean']:.2f}",
            "details": {
                "statistics": {
                    "mean": analysis["mean"],
                    "max": analysis["max"],
                    "min": analysis["min"]
                },
                "insights": analysis["insights"]
            },
            "timestamp": datetime.now().isoformat()
        }
        
        # Share final report
        resource_id = self.share_resource("final_report", report)
        
        # Publish completion
        self.publish_result("report_generated", {
            "resource_id": resource_id,
            "title": report["title"]
        })
        
        return report


class ValidationAgent(MCPAgent):
    """Agent that validates outputs from other agents"""
    
    def __init__(self):
        super().__init__(
            agent_id="validator",
            role="Validation",
            capabilities=["quality_check", "verification", "compliance"]
        )
    
    async def process_task(self, task: str) -> Dict[str, Any]:
        """Validate report quality"""
        print(f"\n✅ [{self.agent_id}] Validating outputs for: {task}")
        
        # Get all messages to validate the workflow
        all_messages = self.get_messages()
        
        validation = {
            "task": task,
            "workflow_complete": False,
            "steps_validated": [],
            "issues": []
        }
        
        # Check for required steps
        required_steps = ["data_collected", "analysis_complete", "report_generated"]
        
        for step in required_steps:
            step_messages = [m for m in all_messages if m["type"] == step]
            if step_messages:
                validation["steps_validated"].append(step)
                print(f"  ✓ {step} validated")
            else:
                validation["issues"].append(f"Missing step: {step}")
                print(f"  ✗ {step} missing")
        
        validation["workflow_complete"] = len(validation["steps_validated"]) == len(required_steps)
        
        # Publish validation results
        self.publish_result("validation_complete", validation)
        
        return validation

## Part 4: MCP Coordinator

The coordinator orchestrates the multi-agent workflow using MCP protocol.

In [ ]:
class MCPCoordinator:
    """Coordinates multi-agent workflows using MCP"""
    
    def __init__(self):
        self.agents: Dict[str, MCPAgent] = {}
        self.message_bus = message_bus
    
    def register_agent(self, agent: MCPAgent):
        """Register an agent with the coordinator"""
        self.agents[agent.agent_id] = agent
        print(f"✅ Registered agent: {agent.agent_id} ({agent.role})")
    
    async def execute_workflow(self, task: str, agent_sequence: List[str]):
        """Execute a multi-agent workflow"""
        print(f"\n{'='*80}")
        print(f"🚀 Starting MCP Multi-Agent Workflow")
        print(f"📋 Task: {task}")
        print(f"🔄 Agent Sequence: {' → '.join(agent_sequence)}")
        print(f"{'='*80}\n")
        
        results = {}
        
        for agent_id in agent_sequence:
            if agent_id not in self.agents:
                print(f"❌ Agent {agent_id} not found")
                continue
            
            agent = self.agents[agent_id]
            result = await agent.process_task(task)
            results[agent_id] = result
            
            # Small delay to simulate processing time
            await asyncio.sleep(0.3)
        
        print(f"\n{'='*80}")
        print(f"✨ Workflow Complete")
        print(f"{'='*80}\n")
        
        return results
    
    def get_message_history(self) -> List[Dict]:
        """Get all messages from the bus"""
        return self.message_bus.messages
    
    def get_shared_resources(self) -> Dict[str, Any]:
        """Get all shared resources"""
        return self.message_bus.resources
    
    def display_communication_log(self):
        """Display the MCP communication log"""
        print("\n📡 MCP Communication Log:")
        print("="*80)
        
        for msg in self.message_bus.messages:
            timestamp = msg["timestamp"].split("T")[1].split(".")[0]
            print(f"[{timestamp}] {msg['agent_id']:20} | {msg['type']:25} | {str(msg['content'])[:50]}")
        
        print("="*80)

## Part 5: Running the Multi-Agent System

Let's create agents and execute workflows using MCP communication.

In [ ]:
# Create the coordinator
coordinator = MCPCoordinator()

# Create and register agents
data_collector = DataCollectorAgent()
analyzer = AnalysisAgent()
report_gen = ReportGeneratorAgent()
validator = ValidationAgent()

coordinator.register_agent(data_collector)
coordinator.register_agent(analyzer)
coordinator.register_agent(report_gen)
coordinator.register_agent(validator)

### Example 1: Complete Data Pipeline

In [ ]:
# Execute a complete workflow
task = "Quarterly Sales Analysis"
agent_sequence = ["data_collector", "analyzer", "report_generator", "validator"]

results = await coordinator.execute_workflow(task, agent_sequence)

In [ ]:
# Display the communication log
coordinator.display_communication_log()

In [ ]:
# View shared resources
print("\n💾 Shared Resources via MCP:")
print("="*80)
for resource_id, data in coordinator.get_shared_resources().items():
    print(f"\n📦 Resource: {resource_id}")
    print(json.dumps(data, indent=2))

### Example 2: Parallel Agent Execution

MCP allows agents to work independently and share results.

In [ ]:
async def parallel_data_collection():
    """Multiple data collectors working in parallel"""
    
    # Create multiple data collectors
    collector1 = DataCollectorAgent()
    collector1.agent_id = "data_collector_1"
    
    collector2 = DataCollectorAgent()
    collector2.agent_id = "data_collector_2"
    
    # Run in parallel
    tasks = [
        collector1.process_task("Dataset A"),
        collector2.process_task("Dataset B")
    ]
    
    results = await asyncio.gather(*tasks)
    
    print("\n✅ Parallel collection complete!")
    return results

# Run parallel collection
parallel_results = await parallel_data_collection()

### Example 3: Agent Discovery via MCP

Agents can discover other agents and their capabilities using MCP.

In [ ]:
def discover_agents():
    """Discover all registered agents and their capabilities"""
    print("\n🔍 Agent Discovery via MCP:")
    print("="*80)
    
    registration_messages = [m for m in message_bus.messages 
                            if m["type"] == "agent_registered"]
    
    for msg in registration_messages:
        agent_id = msg["agent_id"]
        content = msg["content"]
        print(f"\n🤖 Agent: {agent_id}")
        print(f"   Role: {content['role']}")
        print(f"   Capabilities: {', '.join(content['capabilities'])}")
    
    print("\n" + "="*80)

discover_agents()

### Example 4: Real-time Agent Communication

Watch agents communicate in real-time via MCP.

In [ ]:
async def realtime_communication_demo():
    """Demonstrate real-time inter-agent communication"""
    
    print("\n🔴 LIVE: Real-time Agent Communication Demo")
    print("="*80)
    
    # Create a new message bus for clean demo
    demo_bus = MessageBus()
    
    # Create agents with the demo bus
    class DemoAgent(MCPAgent):
        def __init__(self, agent_id, role):
            self.agent_id = agent_id
            self.role = role
            self.capabilities = []
            self.message_bus = demo_bus
    
    agent_a = DemoAgent("AgentA", "Requester")
    agent_b = DemoAgent("AgentB", "Processor")
    agent_c = DemoAgent("AgentC", "Validator")
    
    # Simulate conversation
    print("\n📤 AgentA: Requesting data processing")
    agent_a.publish_result("request", {"task": "Process customer data"})
    await asyncio.sleep(0.5)
    
    print("📥 AgentB: Received request, processing...")
    messages = agent_b.get_messages("request")
    if messages:
        agent_b.publish_result("processing", {"status": "in_progress"})
    await asyncio.sleep(0.5)
    
    print("📤 AgentB: Sharing processed results")
    agent_b.share_resource("processed_data", {"result": "Customer insights"})
    await asyncio.sleep(0.5)
    
    print("📥 AgentC: Validating results")
    resource_msgs = agent_c.get_messages("resource_shared")
    if resource_msgs:
        resource_id = resource_msgs[-1]["content"]["resource_id"]
        data = agent_c.access_resource(resource_id)
        agent_c.publish_result("validated", {"status": "approved"})
    
    print("\n✅ Communication sequence complete!")
    print("="*80)
    
    return demo_bus.messages

demo_messages = await realtime_communication_demo()

## Part 6: MCP Protocol Benefits

### Key Advantages:

1. **Standardized Communication**: All agents use the same protocol
2. **Resource Sharing**: Agents can share data, tools, and context
3. **Loose Coupling**: Agents don't need to know about each other's internals
4. **Discoverability**: Agents can discover capabilities of other agents
5. **Scalability**: Easy to add new agents without changing existing ones
6. **Interoperability**: Different AI systems can work together

## Part 7: Advanced MCP Patterns

In [ ]:
class MCPToolRegistry:
    """Registry for MCP tools that agents can expose and discover"""
    
    def __init__(self):
        self.tools: Dict[str, Dict] = {}
    
    def register_tool(self, agent_id: str, tool_name: str, tool_func, description: str):
        """Register a tool exposed by an agent"""
        tool_id = f"{agent_id}.{tool_name}"
        self.tools[tool_id] = {
            "agent_id": agent_id,
            "name": tool_name,
            "function": tool_func,
            "description": description
        }
        print(f"🔧 Registered tool: {tool_id}")
    
    def discover_tools(self, capability: Optional[str] = None) -> List[Dict]:
        """Discover available tools"""
        if capability:
            return [t for t in self.tools.values() 
                   if capability.lower() in t["description"].lower()]
        return list(self.tools.values())
    
    def call_tool(self, tool_id: str, *args, **kwargs):
        """Call a registered tool"""
        if tool_id in self.tools:
            return self.tools[tool_id]["function"](*args, **kwargs)
        raise ValueError(f"Tool {tool_id} not found")

# Create global tool registry
tool_registry = MCPToolRegistry()

# Example: Register some tools
def calculate_mean(data: List[float]) -> float:
    return sum(data) / len(data)

def format_report(title: str, data: Dict) -> str:
    return f"# {title}\n\n{json.dumps(data, indent=2)}"

tool_registry.register_tool("analyzer", "calculate_mean", calculate_mean, "Calculate mean of data")
tool_registry.register_tool("report_generator", "format_report", format_report, "Format data as report")

print("\n🔍 Available MCP Tools:")
for tool in tool_registry.discover_tools():
    print(f"  - {tool['agent_id']}.{tool['name']}: {tool['description']}")

## Summary

This notebook demonstrated:

1. **MCP-based Agent Architecture**: Agents as both MCP clients and servers
2. **Message Bus**: Central communication hub using MCP protocol
3. **Resource Sharing**: Agents sharing data and results via MCP resources
4. **Tool Discovery**: Agents discovering and using each other's capabilities
5. **Workflow Orchestration**: Coordinator managing multi-agent workflows
6. **Real-time Communication**: Live agent-to-agent messaging
7. **Parallel Execution**: Multiple agents working simultaneously

### Real-world Applications:

- **Data Processing Pipelines**: Multiple specialized agents for ETL workflows
- **Autonomous Systems**: Agents coordinating in robotics or IoT
- **AI Assistants**: Multiple specialized AI agents collaborating
- **Microservices**: Agent-based microservice architectures
- **Distributed Computing**: Agents working across different systems

### Next Steps:

1. Integrate with real MCP servers and clients
2. Add authentication and security to MCP communication
3. Implement persistent message storage
4. Add monitoring and observability
5. Scale to distributed agent systems